# Data Clean

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

PATH = '/home/kai/data/kaggle/homecredit/'
inst0 = pd.read_csv(PATH + 'installments_payments.csv')

In [2]:
inst = inst0.fillna(0)

In [3]:
inst.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT'],
      dtype='object')

In [4]:
g = inst.copy()
gp_col = ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER']

this_gp_col = gp_col + ['NUM_INSTALMENT_VERSION', 'DAYS_INSTALMENT']
tmp = g[this_gp_col + ['AMT_INSTALMENT']].groupby(this_gp_col)[['AMT_INSTALMENT']].mean().reset_index()
print('AMT_INST')

this_gp_col = gp_col + ['DAYS_INSTALMENT']
g = g.merge(tmp[this_gp_col+['AMT_INSTALMENT']].groupby(this_gp_col)[['AMT_INSTALMENT']].sum().reset_index().rename(columns={'AMT_INSTALMENT':'new_AMT_INSTALMENT'}), on=this_gp_col, how='left')
g = g.drop('AMT_INSTALMENT', axis=1).rename(columns={'new_AMT_INSTALMENT': 'AMT_INSTALMENT'})
print('new_AMT')

AMT_INST
new_AMT


In [5]:
group = g.copy()

this_gp_col = gp_col + ['NUM_INSTALMENT_VERSION']
group = group.merge(g.groupby(this_gp_col).size().reset_index().rename(columns={0:'num'}), on=this_gp_col, how='left')
print('num')

group = group.merge(group[gp_col+['num']].groupby(gp_col)[['num']].nunique().reset_index().rename(columns={'num':'equal'}), on=gp_col, how='left')
print('equal')

group = group.merge(group[this_gp_col].groupby(gp_col)[['NUM_INSTALMENT_VERSION']].max().reset_index().rename(columns={'NUM_INSTALMENT_VERSION':'max'}), on=gp_col, how='left')
print('max')

num
equal
max


In [6]:
final = group[~((group['num']>1) & (group['equal']==1) & (group['NUM_INSTALMENT_VERSION']!=group['max']))]
print('final')

final


In [7]:
import os
if not os.path.exists(PATH + 'inter/tmp'):
    os.mkdir(PATH + 'inter/tmp')
types = {
    'DAYS_INSTALMENT': 'int64',
    'DAYS_ENTRY_PAYMENT': 'int64'
}
final.drop(['num', 'equal', 'max', 'NUM_INSTALMENT_VERSION'], axis=1).astype(types).to_pickle(PATH + '/inter/tmp/inst.pkl')

 # test code

In [35]:
tmp = inst.groupby(gp_col)[['NUM_INSTALMENT_VERSION']].nunique().reset_index()

ttmp = inst[gp_col+['NUM_INSTALMENT_VERSION']].groupby(gp_col+['NUM_INSTALMENT_VERSION']).size().reset_index().groupby(gp_col)[0].nunique().reset_index()
tmp = tmp.merge(ttmp, on=gp_col, how='left')

ttmp = inst.groupby(gp_col)[['DAYS_INSTALMENT']].count().reset_index()
tmp = tmp.merge(ttmp, on=gp_col, how='left')

In [10]:
p = [2426357, 15]
inst[(inst['SK_ID_PREV']==p[0]) & (inst['NUM_INSTALMENT_NUMBER']==p[1])].sort_values('NUM_INSTALMENT_VERSION')

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
408458,2426357,121033,1.0,15,-994.0,-956.0,8337.060,656.595
1292876,2426357,121033,1.0,15,-994.0,-1001.0,8337.060,14.400
1526192,2426357,121033,1.0,15,-994.0,-977.0,8337.060,13500.000
3197210,2426357,121033,1.0,15,-994.0,-988.0,8337.060,8338.500
3552064,2426357,121033,1.0,15,-994.0,-1001.0,8337.060,8337.060
408459,2426357,121033,2.0,15,-994.0,-956.0,22509.495,656.595
1292877,2426357,121033,2.0,15,-994.0,-1001.0,22509.495,14.400
1526193,2426357,121033,2.0,15,-994.0,-977.0,22509.495,13500.000
3197211,2426357,121033,2.0,15,-994.0,-988.0,22509.495,8338.500
3552065,2426357,121033,2.0,15,-994.0,-1001.0,22509.495,8337.060


In [59]:
p = [2426357, 15]
final[(final['SK_ID_PREV']==p[0]) & (final['NUM_INSTALMENT_NUMBER']==p[1])]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,new_AMT_INSTALMENT,num,equal,max
408459,2426357,121033,2.0,15,-994.0,-956.0,22509.495,656.595,30846.555,5,1,2.0
1292877,2426357,121033,2.0,15,-994.0,-1001.0,22509.495,14.400,30846.555,5,1,2.0
1526193,2426357,121033,2.0,15,-994.0,-977.0,22509.495,13500.000,30846.555,5,1,2.0
3197211,2426357,121033,2.0,15,-994.0,-988.0,22509.495,8338.500,30846.555,5,1,2.0
3552065,2426357,121033,2.0,15,-994.0,-1001.0,22509.495,8337.060,30846.555,5,1,2.0


In [60]:
p = [2426357, 15]
payment_series[(payment_series['SK_ID_PREV']==p[0]) & (payment_series['NUM_INSTALMENT_NUMBER']==p[1])]

,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,LATE_DAYS_SIGN,AMT_PAYMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,DAYS_INSTALMENT,SK_ID_CURR,AMT_CUM_PAYMENT,AMT_LATE_PAYMENT,RATE_LATE_PAYMENT
10600979,2426357,15,2.0,-0.0,8351.460,-994.0,22509.495,-994.0,121033,8351.460,14158.035,1.695074
10600980,2426357,15,2.0,6.0,8338.500,-988.0,22509.495,-994.0,121033,16689.960,5819.535,0.348664
10600981,2426357,15,2.0,17.0,13500.000,-977.0,22509.495,-994.0,121033,30189.960,-7680.465,-0.254396
10600982,2426357,15,2.0,38.0,656.595,-956.0,22509.495,-994.0,121033,30846.555,-8337.060,-0.270266


In [58]:
tmp[tmp['NUM_INSTALMENT_VERSION'] == 2].sort_values('DAYS_INSTALMENT', ascending=False)

,SK_ID_PREV,NUM_INSTALMENT_NUMBER,NUM_INSTALMENT_VERSION,0,DAYS_INSTALMENT
10045364,2426357,15,2,1,10
2079363,1282551,8,2,1,8
11626945,2660511,6,2,1,8
4953514,1676214,9,2,1,8
1780379,1239348,23,2,1,8
1770933,1238449,10,2,1,8
10836139,2543281,14,2,1,8
6196539,1861225,5,2,1,6
1198410,1159357,7,2,1,6
2653066,1360999,18,2,1,6


# feature code

In [8]:
from sklearn.linear_model import LinearRegression

def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / (df[col2] + 1)

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join(gp_col + [col])})
    return group

def count(df, gp_col, col):
    group = df[[gp_col, col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index()

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index()

def linear(df, gp_col, x_col_name, y_col_name):
    r = []
    for i, x in df.groupby(gp_col):
        lg = LinearRegression()
        lg1 = LinearRegression()
        
        lg.fit(x[[x_col_name]], x[[y_col_name]])
        x_max = max(x[x_col_name])
        x_min = min(x[x_col_name])
        lg1.fit((x[[x_col_name]]-x_max)/(x_max-x_min+1), x[[y_col_name]])
        r.append([i, lg.coef_[0][0], lg1.coef_[0][0]])
    tmpdf = pd.DataFrame(r, columns=[gp_col, x_col_name+'_lg_'+y_col_name, x_col_name+'_normallg_'+y_col_name])
    return tmpdf

def feature_in_time_window(df, gp_col, col, func, agg_fun=None, n=None, time_col=None):
    tmp = None
    _df = df.copy()
    if n is not None:
        _df = _df[_df[time_col] >= n]
    if agg_fun is None:
        tmp = func(_df, gp_col, col)
    else:
        tmp = func(_df, gp_col, col, agg_fun)
    columns = [str(abs(n))+'_'+x for x in set(tmp.columns) - set(gp_col)]
    tmp.columns = columns
    del _df
    gc.collect()
    return tmp

def area_under_curve(df, gp_col, x, y):
    gp = df.copy()
    gp_max = gp.groupby(gp_col)[[x]].max().reset_index().rename(columns={x:'max'})
    gp_min = gp.groupby(gp_col)[[x]].min().reset_index().rename(columns={x:'min'})
    gp = gp.merge(gp_max)
    gp = gp.merge(gp_min)
    gp['normal_x'] = (gp[x]-gp['max']) / (gp['max']-gp['min']+1)
    
    group = gp.groupby(gp_col)
    gp['tmp'] = (group[y].shift(-1)+gp[y]) * (group[x].shift(-1)-gp[x]) / 2
    gp['tmp_normal'] = (group[y].shift(-1)+gp[y]) * (group['normal_x'].shift(-1)-gp['normal_x']) / 2
    return gp.groupby(gp_col).agg({'tmp':'sum', 'tmp_normal':'sum'}).reset_index().rename(columns={'tmp':x+'_area_'+y, 'tmp_normal':x+'_normalarea_'+y})

'done'

'done'

In [14]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

PATH = '/home/kai/data/kaggle/homecredit/'

inst0 = pd.read_pickle(PATH + '/inter/tmp/inst.pkl')

new = inst0.groupby(['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'SK_ID_CURR'])[['AMT_INSTALMENT']].max().reset_index()
new['AMT_PAYMENT'] = 0
new['DAYS_ENTRY_PAYMENT'] = new['DAYS_INSTALMENT']
inst = pd.concat([inst0, new])

inst['EARLY_DAYS'] = inst['DAYS_INSTALMENT'] - inst['DAYS_ENTRY_PAYMENT']
inst['LATE_DAYS'] = -inst['EARLY_DAYS']
inst['LATE_DAYS_SIGN'] = (np.sign(inst['LATE_DAYS']) > 0) * inst['LATE_DAYS']

gp_col = ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT']
payment_series = inst.groupby(gp_col+['LATE_DAYS_SIGN']).agg({'AMT_PAYMENT': 'sum', 'DAYS_ENTRY_PAYMENT': 'max', 'AMT_INSTALMENT': 'max', 'SK_ID_CURR': 'max'}).reset_index()
payment_series['AMT_CUM_PAYMENT'] = payment_series.groupby(gp_col)[['AMT_PAYMENT']].cumsum()
payment_series['AMT_LATE_PAYMENT'] = payment_series['AMT_INSTALMENT'] - payment_series['AMT_CUM_PAYMENT']
payment_series['AMT_LATE_PAYMENT'] = payment_series['AMT_LATE_PAYMENT'] * (abs(payment_series['AMT_LATE_PAYMENT'])>1e-4)
payment_series['RATE_LATE_PAYMENT'] = payment_series['AMT_LATE_PAYMENT'] / (payment_series['AMT_CUM_PAYMENT']+1)
# payment_series['DAYS_ENTRY_PAYMENT'] = payment_series[['DAYS_ENTRY_PAYMENT', 'DAYS_INSTALMENT']].max(axis=1)
payment_series.sort_values('DAYS_ENTRY_PAYMENT', ascending=False)
payment_series

,SK_ID_PREV,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,LATE_DAYS_SIGN,AMT_PAYMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,SK_ID_CURR,AMT_CUM_PAYMENT,AMT_LATE_PAYMENT,RATE_LATE_PAYMENT
0,1000001,1,-268,0,6404.310,-268,6404.310,158271,6404.310,0.000,0.00000
1,1000001,2,-238,0,62039.115,-238,62039.115,158271,62039.115,0.000,0.00000
2,1000002,1,-1600,0,6264.000,-1600,6264.000,101962,6264.000,0.000,0.00000
3,1000002,2,-1570,0,6264.000,-1570,6264.000,101962,6264.000,0.000,0.00000
4,1000002,3,-1540,0,6264.000,-1540,6264.000,101962,6264.000,0.000,0.00000
5,1000002,4,-1510,0,18443.565,-1510,18443.565,101962,18443.565,0.000,0.00000
6,1000003,1,-94,0,4951.350,-94,4951.350,252457,4951.350,0.000,0.00000
7,1000003,2,-64,0,4951.350,-64,4951.350,252457,4951.350,0.000,0.00000
8,1000003,3,-34,0,4951.350,-34,4951.350,252457,4951.350,0.000,0.00000
9,1000004,1,-862,0,3391.110,-862,3391.110,260094,3391.110,0.000,0.00000


In [18]:
payment_series['AMT_LATE_PAYMENT'].min()

-2753704.935

In [15]:
payment_series[(payment_series['SK_ID_PREV']==1000005) & (payment_series['NUM_INSTALMENT_NUMBER']==9)]

,SK_ID_PREV,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,LATE_DAYS_SIGN,AMT_PAYMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,SK_ID_CURR,AMT_CUM_PAYMENT,AMT_LATE_PAYMENT,RATE_LATE_PAYMENT
25,1000005,9,-1448,0,2.790,-1448,14713.605,176456,2.790,14710.815,3881.48153
26,1000005,9,-1448,3,14710.815,-1445,14713.605,176456,14713.605,-0.000,-0.00000


In [11]:
inst[(inst['SK_ID_PREV']==1000005) & (inst['NUM_INSTALMENT_NUMBER']==9)]

,AMT_INSTALMENT,AMT_PAYMENT,DAYS_ENTRY_PAYMENT,DAYS_INSTALMENT,NUM_INSTALMENT_NUMBER,SK_ID_CURR,SK_ID_PREV,EARLY_DAYS,LATE_DAYS,LATE_DAYS_SIGN
1740975,14713.605,14710.815,-1445,-1448,9,176456,1000005,-3,3,3
2618814,14713.605,2.790,-1484,-1448,9,176456,1000005,36,-36,0
24,14713.605,0.000,-1448,-1448,9,176456,1000005,0,0,0


In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')

df = pd.concat([train[['SK_ID_CURR']], test[['SK_ID_CURR']]])
print(df.shape)

(356255, 1)


In [4]:
gp_col = ['SK_ID_CURR', 'SK_ID_PREV', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT']
tmp_df = inst.groupby(gp_col).size().reset_index()[gp_col]

for func in [area_under_curve, linear]:
    print(func)
    for x in ['AMT_LATE_PAYMENT', 'RATE_LATE_PAYMENT']:
        print(x)
        tmp_df = tmp_df.merge(func(payment_series, gp_col, 'DAYS_ENTRY_PAYMENT', x), on=gp_col, how='left')
        print(tmp_df.shape)
    
merge_col = ['SK_ID_CURR']
agg_fun = ['sum', 'mean', 'median', 'max', 'min']
for x in tmp_df.columns:
    if x not in gp_col:
        print(x)
        df = df.merge(numerical(tmp_df, merge_col, x, agg_fun))
        print('num\t', df.shape)
    
for x in ['EARLY_DAYS', 'LATE_DAYS']:
    print(x)
    df = df.merge(positive_count(inst, merge_col, x), on=merge_col, how='left')
    print('positive count\t', tmp_df.shape)

for func in [area_under_curve, linear]:
    print(func)
    for x in ['AMT_LATE_PAYMENT', 'RATE_LATE_PAYMENT', 'LATE_DAYS_SIGN']:
        print(x)
        df = df.merge(func(payment_series, merge_col, 'DAYS_ENTRY_PAYMENT', x), on=merge_col, how='left')
        print(df.shape)

<function area_under_curve at 0x7f3376dcc8c8>
AMT_LATE_PAYMENT
(12862309, 6)
RATE_LATE_PAYMENT
(12862309, 8)
<function linear at 0x7f3376dcc840>
AMT_LATE_PAYMENT


KeyboardInterrupt: 

In [ ]:
df.to_pickle(PATH + 'inter/linear/newnewlinearinst2curr.pkl')

In [ ]:
set('a')

In [ ]:
list_todo = ['']